In [1]:
!pip install pandas

In [0]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
import torch.optim as optim
import torch.nn as nn
import datetime as dtm
from collections import defaultdict
import operator
from itertools import islice
import pdb
import pprint as pp
import pprint as pp
from tqdm import tqdm

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
main_path = 'drive/My Drive/Colab Notebooks/Shopee_one'

In [0]:
rawdf = pd.read_csv(os.path.join(main_path, 'order_brush_order.csv'))
rawdf = rawdf.sort_values('event_time', ascending=1)

In [6]:
header_list = list(rawdf.columns.values)
print(header_list)

['orderid', 'shopid', 'userid', 'event_time']


In [0]:
listof_shop_id = [rawdf.iloc[idx]['shopid'] for idx in range(len(rawdf['shopid']))]
listof_user_id = [rawdf.iloc[idx]['userid'] for idx in range(len(rawdf['userid']))]

In [0]:
uniq_shop_id = list(set(listof_shop_id))
uniq_user_id = list(set(listof_user_id))

In [9]:
print(listof_shop_id[:10])
print(listof_user_id[:10])

[6042309, 104804492, 8715449, 190969466, 2859407, 94479614, 65551316, 147941492, 54615708, 127236302]
[97707522, 97707522, 97707522, 170182475, 12532131, 12532131, 12532131, 195903090, 128755891, 128755891]


In [10]:
len(uniq_shop_id), len(listof_shop_id)

(18770, 222750)

In [11]:
rawdf.head(10)

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05
190853,31075205798265,94479614,12532131,2019-12-27 00:00:05
30633,31075205798263,65551316,12532131,2019-12-27 00:00:05
2338,31075208826809,147941492,195903090,2019-12-27 00:00:08
112662,31075207143538,54615708,128755891,2019-12-27 00:00:08
87378,31075207143537,127236302,128755891,2019-12-27 00:00:08


In [0]:
shop_records = {s : {h : [] for h in header_list} for s in uniq_shop_id}

In [13]:
""" To get a dictionary with unique shopids as keys and their activities (orders with corresponding buyer, and eventime which are also in dictionary container)"""


# stopper = 10
min_event = dtm.datetime.strptime(rawdf.iloc[0]['event_time'],"%Y-%m-%d %H:%M:%S")
max_event = dtm.datetime.strptime(rawdf.iloc[0]['event_time'],"%Y-%m-%d %H:%M:%S")

for idx in tqdm(range(len(rawdf))):
  for h in header_list:
    if h == 'event_time':
      shop_records[rawdf.iloc[idx]['shopid']][h] = shop_records[rawdf.iloc[idx]['shopid']][h] + [dtm.datetime.strptime(rawdf.iloc[idx]['event_time'],"%Y-%m-%d %H:%M:%S")]
      if min_event > shop_records[rawdf.iloc[idx]['shopid']][h][-1]:
        min_event = shop_records[rawdf.iloc[idx]['shopid']][h][-1]
      if max_event < shop_records[rawdf.iloc[idx]['shopid']][h][-1]:
        max_event = shop_records[rawdf.iloc[idx]['shopid']][h][-1]
    else:
      shop_records[rawdf.iloc[idx]['shopid']][h] = shop_records[rawdf.iloc[idx]['shopid']][h] + [str(rawdf.iloc[idx][h])]
  # stopper -= 1
  # pp.pprint(shop_records[rawdf.iloc[idx]['shopid']])
  # if stopper < 1:
  #   break

# del stopper
print([i for i in (min_event, max_event)])

100%|██████████| 222750/222750 [09:37<00:00, 385.92it/s]

[datetime.datetime(2019, 12, 27, 0, 0), datetime.datetime(2019, 12, 31, 23, 59, 56)]


In [0]:
# Only work for list of datetime objects with ordered from earliest to latest, see csv panda.dataframe sorting script in cell 
def min_max_datetime(event_list: list):
  in_range = True
  one_hour_idxs = []
  temp_hour = []
  event_start = event_list[0]
  event_end = event_list[-1]
  if len(event_list) < 3:
    in_range = False
  if in_range:
    for idx0, timestamp in enumerate(event_list):
      for idx1, endstamp in enumerate(event_list[idx0 + 1:]):
        # pdb.set_trace()
        if ((endstamp - timestamp).total_seconds() >= 3600):
          if idx0 != idx0 + idx1 and idx1 > 1:
            temp_hour = [(idx0, idx0 + idx1)]
          if len(one_hour_idxs) > 0 and len(temp_hour) != 0:
            if one_hour_idxs[-1][-1] == temp_hour[-1][-1]:
              temp_hour = []
          break
      if len(temp_hour) == 0 and (idx1 + idx0 + 2 == len(event_list)) and len(event_list) - idx0 > 2:
        temp_hour = temp_hour + [(idx0, len(event_list) - 1)]
        if temp_hour[0][-1] - temp_hour[0][0] + 1 > 2:
          one_hour_idxs = one_hour_idxs + temp_hour
        break
      one_hour_idxs = one_hour_idxs + temp_hour
      temp_hour = []
  if len(one_hour_idxs) == 0 and in_range:
    if len(event_list) > 2:
          one_hour_idxs = one_hour_idxs + [(0, len(event_list) - 1)]
  return one_hour_idxs, in_range

In [0]:
# Only works for this problem with the keys of dict using original csv column headers in codecell #5
def cheater_checker(shop: dict):
  """ To determine if a uniqe shop is a cheater and find the corresponding buyer that boost the sales"""
  check_idxs, go_check = min_max_datetime(shop['event_time'])
  suspect_buyer = '&0'
  for ids in check_idxs:
    uniq_buy_hourly = list(set(shop['userid'][ids[0]:ids[1] + 1]))
    buyer_list_hourly = shop['userid'][ids[0]:ids[1] + 1]
    if len(buyer_list_hourly)/len(uniq_buy_hourly) >= 3.0:
      suspect_buyer = '&'.join([suspect_buyer, str(max(buyer_list_hourly, key=buyer_list_hourly.count))])
  return suspect_buyer[3:] if len(suspect_buyer) > 2 else suspect_buyer[1]

In [86]:
shop_final_data = {s : '' for s in uniq_shop_id}
for i in tqdm(range(int(len(uniq_shop_id)))):
  shop_final_data[uniq_shop_id[i]] = cheater_checker(shop_records[uniq_shop_id[i]])

100%|██████████| 18770/18770 [00:01<00:00, 10750.72it/s]


In [0]:
submission = pd.DataFrame(data={'shopid': list(shop_final_data.keys()),
                                'userid': list(shop_final_data.values())})

In [0]:
submission.to_csv(path_or_buf=os.path.join(main_path, 'submission.csv'), index=False)

In [0]:
asutest = {'event_time': [dtm.datetime(2019, 12, 29, 20, 37, 35),
                          dtm.datetime(2019, 12, 29, 20, 38, 56),
                          dtm.datetime(2019, 12, 29, 20, 39, 50),
                          dtm.datetime(2019, 12, 29, 20, 40, 10),
                          dtm.datetime(2019, 12, 29, 20, 41, 39),
                          dtm.datetime(2019, 12, 29, 20, 42, 39),
                          dtm.datetime(2019, 12, 29, 20, 43, 39),
                          dtm.datetime(2019, 12, 29, 20, 44, 39)],
           'orderid': [0,1,2,3,4,5,6,7],
           'shopid': [69696969]*len([0,1,2,3,4,5,6,7]),
           'userid': ['fak1']*len([0,1,2]) + ['fak2']*len([3,4,5,6,7])}

asutest2 = {'event_time': [dtm.datetime(2019, 12, 29, 20, 37, 35),
                          dtm.datetime(2019, 12, 29, 20, 38, 56),
                          dtm.datetime(2019, 12, 29, 20, 39, 50),
                          dtm.datetime(2019, 12, 29, 20, 40, 10),
                          dtm.datetime(2019, 12, 29, 20, 41, 39),
                          dtm.datetime(2019, 12, 31, 20, 42, 39),
                          dtm.datetime(2019, 12, 31, 20, 43, 39),
                          dtm.datetime(2019, 12, 31, 20, 44, 39)],
           'orderid': [0,1,2,3,4,5,6,7],
           'shopid': [69696969]*len([0,1,2,3,4,5,6,7]),
           'userid': ['fak1']*len([0,1,2]) + ['fak2']*len([3,4]) + ['fak3']*len([5,6,7])}

In [114]:
cheater_checker(asutest)

'fak2'

In [115]:
cheater_checker(asutest2)

'fak3'